In [1]:
import numpy as n
import pandas as p
import matplotlib.pyplot as graph
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [31]:
class NeuralNetwork:
  def __init__(self, data, target, lRate = 0.001):
    self.data = n.array(data)
    self.weightSequence = []
    self.biasSequence = []
    self.activations = []
    self.y = n.array(target)
    self.lr = lRate

  def softMax(self, z):
    softMax = n.exp(-z)/n.sum(n.exp(-z))
    return softMax

  def layer(self, units):
    size = self.data.shape[1]

    if len(self.weightSequence) != 0:
      size = self.weightSequence[-1].shape[0]

    weightLayer = n.array([n.array(n.random.random(size)/10000) for i in range(units)])
    biasLayer = n.array([0.0 for i in range(units)])

    self.weightSequence.append(weightLayer)
    self.biasSequence.append(biasLayer)

  def forwardProp(self, data):
    self.activations = []
    weights = self.weightSequence
    bias = self.biasSequence
    layerCount = len(weights)

    layerInput = data

    for i in range(layerCount):
      w = weights[i]
      b = bias[i]

      layerOutput = n.matmul(layerInput, w.T) + b
      self.activations.append(layerOutput)
      layerInput = layerOutput

    return layerInput[0] if len(layerInput) == 1 else layerInput

  def backProp(self):
    self.forwardProp(self.data)
    # print("\n",self.activations)
    m = self.data.shape[0]
    y_cap = self.softMax(self.activations[-1])
    error = y_cap - self.y
    # print(y_cap, self.y)
    # print("error",error)
    # print("activation",self.activations[-2])

    dj_dw = n.dot(self.activations[-2].T, error)/m
    dj_db = n.sum(error, axis = 0, keepdims=True)/m
    # print(dj_dw, dj_db)


    output_weights = self.weightSequence[-1]
    output_bias = self.biasSequence[-1]

    # print(output_weights, output_bias)

    output_weights -= self.lr*dj_dw.T
    output_bias -= self.lr*dj_db[0]

    for i in range(len(self.weightSequence) - 2, -1, -1):
      # print(i, "\n--------------")

      dj_dyi = error
      prevActivation = self.activations[i-1] if i-1 >= 0 else self.data
      # print(dj_dyi, "\n\n", self.weightSequence[i+1], "\n\n", prevActivation)
      error = n.matmul(dj_dyi, self.weightSequence[i+1])

      dj_dw = n.matmul(error.T, prevActivation)/m
      dj_db = n.sum(error, axis = 0)/m

      self.weightSequence[i] -= self.lr*dj_dw
      self.biasSequence[i] -= self.lr*dj_db

  def trainModel(self, iter = 1000, lr = 0.0001):
      self.lr = lr
      for i in range(iter):
        self.backProp()


In [3]:
dataSheet = p.read_csv(f"gdrive/My Drive/ML WOC/datasheets/nn_train.csv")
data = n.array(dataSheet)

In [ ]:
x = data[:2000, 1:-2]
y = data[:2000, -1].reshape(x.shape[0], 1)


nn = NeuralNetwork(x, y)
nn.layer(128)
nn.layer(64)
nn.layer(32)
nn.layer(10)

# print(nn.weightSequence, "\n")
# print(nn.biasSequence, "\n")

nn.trainModel(5000, 0.0001)

# print(nn.weightSequence, "\n")
# print(nn.biasSequence, "\n")

# nn.forwardProp([[1, 2, 3], [4, 5, 6]])
# print(nn.sigmoid(nn.activations[-1]))
# print("+++++++++++++++++")
# for i in nn.activations:
#   print("\n", i)
# print("\n", nn.weightSequence)
# print(nn.biasSequence)



<ipython-input-31-115f230c2e6a>:11: RuntimeWarning: invalid value encountered in divide
  softMax = n.exp(-z)/n.sum(n.exp(-z))


In [52]:
nn.forwardProp(data[:, 1:-2])
a = nn.activations[-1]
for i in a:
  print(i, n.argmax(i))

[0.55340375 0.55340368 0.55340369 0.55340372 0.55340361 0.55340364
 0.55340391 0.55340396 0.55340346 0.5534036 ] 7
[0.55340442 0.55340435 0.55340437 0.55340439 0.55340427 0.5534043
 0.5534046  0.55340465 0.55340412 0.55340427] 7
[0.55340305 0.55340299 0.553403   0.55340303 0.55340293 0.55340295
 0.5534032  0.55340325 0.55340279 0.55340292] 7
[0.55340323 0.55340316 0.55340318 0.5534032  0.5534031  0.55340312
 0.55340338 0.55340343 0.55340296 0.55340309] 7
[0.55340378 0.55340371 0.55340372 0.55340375 0.55340364 0.55340367
 0.55340394 0.553404   0.55340349 0.55340363] 7
[0.55340303 0.55340297 0.55340299 0.55340301 0.55340291 0.55340293
 0.55340318 0.55340323 0.55340277 0.5534029 ] 7
[0.55340472 0.55340465 0.55340466 0.55340469 0.55340457 0.5534046
 0.5534049  0.55340496 0.55340441 0.55340456] 7
[0.5534035  0.55340344 0.55340345 0.55340347 0.55340337 0.5534034
 0.55340366 0.55340371 0.55340323 0.55340336] 7
[0.55340372 0.55340365 0.55340366 0.55340369 0.55340358 0.55340361
 0.55340388 0.55

KeyboardInterrupt: 

In [ ]:
dataSheet_t = p.read_csv(f"gdrive/My Drive/ML WOC/datasheets/nn_test.csv")
data_t = n.array(dataSheet)

In [ ]:
data_test = list(data_t )
data_with_target_test = []
lastColumnIndex = len(data_test[0])

for i in data_test:
  features = []
  prediction = 1 if nn.sigmoid(nn.forwardProp(i[1:-2])) >= 0.5 else 0
  features.append(prediction)
  data_with_target_test.append(features)

data_with_target_test = n.array(data_with_target_test)
df_with_target_test = p.DataFrame(data_with_target_test , columns=["Binary_predictions"])
df_with_target_test.to_csv('nn_test_predictions_binary_labels.csv', index=True)